In [ ]:
open System.IO
open System.Diagnostics

let run name args =
    let info = ProcessStartInfo()
    info.Arguments <- args
    info.FileName <- name
    info.UseShellExecute <- false
    let p = Process.Start(info)
    p.WaitForExit()

let resultsTypeCheckingBenchmark1Path = Path.Combine(__SOURCE_DIRECTORY__, "BenchmarkDotNet.Artifacts\\results\\BenchmarkComparison.TypeCheckingBenchmark1-report.csv")

run "dotnet" "run -c Release --project run_current.fsproj"
let results_current = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "Current Done"
run "dotnet" "run -c Release --project run_40_0_0.fsproj"
let results_40_0_0 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "40.0.0 Done"
run "dotnet" "run -c Release --project run_35_0_0.fsproj"
let results_35_0_0 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "35.0.0 Done"
run "dotnet" "run -c Release --project run_30_0_0.fsproj"
let results_30_0_0 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "30.0.0 Done"
run "dotnet" "run -c Release --project run_25_0_1.fsproj"
let results_25_0_1 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "25.0.1 Done"
run "dotnet" "run -c Release --project run_20_0_1.fsproj"
let results_20_0_1 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "20.0.1 Done"
run "dotnet" "run -c Release --project run_13_0_0.fsproj"
let results_13_0_0 = File.ReadAllText(resultsTypeCheckingBenchmark1Path)
printfn "13.0.0 Done"

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"
#r "nuget: Microsoft.Data.Analysis, 0.18.0"

open Microsoft.Data.Analysis
open Plotly.NET

let df_current = DataFrame.LoadCsvFromString(results_current)
let df_40_0_0 = DataFrame.LoadCsvFromString(results_40_0_0)
let df_35_0_0 = DataFrame.LoadCsvFromString(results_35_0_0)
let df_30_0_0 = DataFrame.LoadCsvFromString(results_30_0_0)
let df_25_0_1 = DataFrame.LoadCsvFromString(results_25_0_1)
let df_20_0_1 = DataFrame.LoadCsvFromString(results_20_0_1)
let df_13_0_0 = DataFrame.LoadCsvFromString(results_13_0_0)

let df = df_current.Append(df_40_0_0.Rows).Append(df_35_0_0.Rows).Append(df_30_0_0.Rows).Append(df_25_0_1.Rows).Append(df_20_0_1.Rows).Append(df_13_0_0.Rows)

In [ ]:
// Data cleanup

let meanColumn = df.Columns.["Mean"]
let allocatedColumn = df.Columns.["Allocated"]

let parseMilliseconds (str: string) =
    if str.Contains(" ms") then
        Single.Parse(str.Replace(" ms", ""))
    elif str.Contains(" s") then
        Single.Parse(str.Replace(" s", "")) * 1000.f
    else
        failwith "Invalid string"

let parseAllocated (str: string) =
    if str.Contains(" MB") then
        Single.Parse(str.Replace(" MB", ""))
    elif str.Contains(" KB") then
        Single.Parse(str.Replace(" KB", "")) / 1024.f
    else
        failwith "Invalid string"

let x =
    [
        "Current (Today)"
        "40.0.0 (6/22/2021)"
        "35.0.0 (4/10/2020)"
        "30.0.0 (6/29/2019)"
        "25.0.1 (9/5/2018)"
        "20.0.1 (2/21/2018)"
        "13.0.0 (6/28/2017)"
    ]

let y1 =
    [
        for i = 0L to meanColumn.Length - 1L do
            meanColumn.[i] :?> string
            |> parseMilliseconds
    ]
let meanData = (x, y1) ||> List.zip |> List.rev
let y2 =
    [
        for i = 0L to allocatedColumn.Length - 1L do
            allocatedColumn.[i] :?> string
            |> parseAllocated
    ]
let allocatedData = (x, y2) ||> List.zip |> List.rev

In [ ]:
// Charts

let meanLine = Chart.Line(meanData, Name="Mean (ms)")
let allocatedLine = Chart.Line(allocatedData, Name="Allocated (MB)")

Chart.Combine([meanLine;allocatedLine])